In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [ ]:
!pip install torch torchvision
!pip install yolov5  # This will install the YOLOv5 package from PyPI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.4/953.4 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.5/660.5 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import zipfile
import os
import io
from google.colab import files
zip_file_path = '/content/drive/My Drive/archive (1).zip'


with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content')  # Extract to the Colab root directory

# Checking the contents of the extracted folder
print(os.listdir('/content'))

Mounted at /content/drive
['.config', 'test', 'train', 'valid', 'README.dataset.txt', 'README.roboflow.txt', 'drive', 'data.yaml', 'sample_data']


In [ ]:
import glob

# Define your classes
classes = [
    "Aluminium foil", "Bottle", "Bottle cap", "Broken glass", "Can",
    "Carton", "Cigarette", "Cup", "Lid", "Other litter",
    "Other plastic", "Paper", "Plastic bag & wrapper", "Plastic bag - wrapper",
    "Plastic container", "Pop tab", "Straw", "Styrofoam piece", "Unlabeled litter"
]


# Define recyclability for each class (1 for recyclable, 0 for non-recyclable)
recyclability = {
    "Aluminium foil": 1, "Bottle": 1, "Bottle cap": 0, "Broken glass": 0, "Can": 1,
    "Carton": 1, "Cigarette": 0, "Cup": 0, "Lid": 0, "Other litter": 0,
    "Other plastic": 1, "Paper": 1, "Plastic bag & wrapper": 0, "Plastic bag - wrapper": 0,
    "Plastic container": 1, "Pop tab": 0, "Straw": 0, "Styrofoam piece": 0, "Unlabeled litter": 0
}


# Path to the directory containing label files
label_dir = '/content/train/labels'

# Get all label files
label_files = glob.glob(os.path.join(label_dir, '*.txt'))

# Initialize a dictionary to hold class ID counts
class_counts = {classname: 0 for classname in classes}

# Iterate over the label files
for file_path in label_files:
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            class_id = int(line.split()[0])
            class_name = classes[class_id]
            class_counts[class_name] += 1

# Print out the counts for each class
for class_name, count in class_counts.items():
    print(f"{class_name}: {count}")

Aluminium foil: 142
Bottle: 1318
Bottle cap: 821
Broken glass: 376
Can: 714
Carton: 662
Cigarette: 2214
Cup: 583
Lid: 252
Other litter: 356
Other plastic: 751
Paper: 369
Plastic bag & wrapper: 2228
Plastic bag - wrapper: 145
Plastic container: 225
Pop tab: 371
Straw: 304
Styrofoam piece: 1419
Unlabeled litter: 0


In [ ]:
# Clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5

# Navigate into the YOLOv5 folder
%cd yolov5

# Install dependencies
!pip install Pillow==9.4.0
!pip install -qr requirements.txt

data_yaml_content = """
train: /content/train/images
val: /content/valid/images
test: /content/test/images

nc: 19
names: ['Aluminium foil', 'Bottle', 'Bottle cap', 'Broken glass', 'Can', 'Carton', 'Cigarette', 'Cup', 'Lid', 'Other litter', 'Other plastic', 'Paper', 'Plastic bag & wrapper', 'Plastic bag - wrapper', 'Plastic container', 'Pop tab', 'Straw', 'Styrofoam piece', 'Unlabeled litter']
"""

with open('/content/data.yaml', 'w') as file:
    file.write(data_yaml_content)

Cloning into 'yolov5'...
remote: Enumerating objects: 16094, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 16094 (delta 0), reused 4 (delta 0), pack-reused 16089
Receiving objects: 100% (16094/16094), 14.82 MiB | 30.66 MiB/s, done.
Resolving deltas: 100% (11023/11023), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 20.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.1.0 which is incompatible.


In [ ]:
!python train.py --img 416 --batch 32 --epochs 75 --data /content/data.yaml --weights yolov5m.pt --cache

2023-12-13 00:46:34.134800: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-13 00:46:34.134846: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-13 00:46:34.134874: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5m.pt, cfg=, data=/content/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=75, batch_size=32, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, pr

In [ ]:
%cd /content/yolov5
!python val.py --img 416 --weights /content/yolov5/runs/train/exp/weights/best.pt --data /content/data.yaml --task test


/content/yolov5
benchmarks.py	 data	     LICENSE	  README.zh-CN.md   setup.cfg	    val.py
CITATION.cff	 detect.py   models	  requirements.txt  train.py	    yolov5m.pt
classify	 export.py   __pycache__  runs		    tutorial.ipynb
CONTRIBUTING.md  hubconf.py  README.md	  segment	    utils
val: data=/content/data.yaml, weights=['/content/yolov5/runs/train/exp/weights/best.pt'], batch_size=32, imgsz=416, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-247-g3f02fde Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)

Fusing layers... 
Model summary: 212 layers, 20925672 parameters, 0 gradients, 48.1 GFLOPs
test: Scanning /content/test/labels... 100 images, 0 backgrounds, 0 corrupt: 100% 100/100 [00:00<00:00, 5656.67it/s]
test: New cache created: /con

In [ ]:
import torch
import os
import torch.quantization
print(os.getcwd())
!python export.py --weights runs/train/exp/weights/best.pt --img-size 416 --batch-size 1 --device 0 --opset 12 --simplify --include onnx
!pip install onnx-tf
!pip install onnx
!pip install onnxruntime
import onnx
from onnx_tf.backend import prepare

# Load your ONNX model
onnx_model = onnx.load("runs/train/exp/weights/best.onnx")  # Update the path

# Prepare the TensorFlow representation
tf_rep = prepare(onnx_model)

# Export the model to a TensorFlow SavedModel
tf_rep.export_graph("exported_model")


/content/yolov5
export: data=data/coco128.yaml, weights=['runs/train/exp/weights/best.pt'], imgsz=[416], batch_size=1, device=0, half=False, inplace=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=True, opset=12, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5 🚀 v7.0-247-g3f02fde Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)

Fusing layers... 
Model summary: 212 layers, 20925672 parameters, 0 gradients, 48.1 GFLOPs

PyTorch: starting from runs/train/exp/weights/best.pt with output shape (1, 10647, 24) (40.3 MB)

ONNX: starting export with onnx 1.15.0...
ONNX: simplifying with onnx-simplifier 0.4.35...
ONNX: export success ✅ 2.8s, saved as runs/train/exp/weights/best.onnx (80.1 MB)

Export complete (6.5s)
Results saved to /content/yolov5/runs/train/exp/weights
Detect:          python detect.py --weights runs/train/exp/weights/best.onnx 

Instructions for updating:
Use `tf.image.resize(...method=ResizeMethod.NEAREST_NEIGHBOR...)` instead.
INFO:absl:Function `__call__` contains input name(s) x, y with unsupported characters which will be renamed to transpose_253_x, onnx_tf_prefix__model_24_mul_11_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.
INFO:absl:Writing fingerprint to exported_model/fingerprint.pb


In [ ]:
import tensorflow as tf

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model("exported_model")
tflite_model = converter.convert()

# Save the model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

from google.colab import files

#files.download('model.tflite')
files.download('exported_model')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>